In [ ]:
# 词袋法可以对文本内容相似性进行度量
# 但是对于两段内容其实差不多但是用的词不一样的就不行了


In [7]:
# nltk主要是进行词性标注的，找到相似性也无法针对具体词汇之间的含义进行度量
# 所以为了找到词汇之间相似度的关系，我们将词汇表示向量化
# 这个其实就可以根据前面三个单词，预测最后一个单词，构成监督学习

In [8]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all')
X, y = news.data, news.target
from bs4 import BeautifulSoup
import nltk, re
# 这nltk扩展包下的还挺麻烦

In [9]:
#定义一个函数名为news_to_sentences
# 将每条新闻中的句子逐一剥离出来，并返回一个句子的列表
def news_to_sentences(news):
    news_text=BeautifulSoup(news).get_text()
    tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences=tokenizer.tokenize(news_text)
    sentences=[]
    for sent in raw_sentences:
        sentences.append(re.sub('[^a-zA-Z]',' ',sent.lower().strip()).split())
    return sentences
sentences=[]

In [10]:
#将长篇新闻文本中的句子剥离出来，用于训练
for x in X:
    sentences+=news_to_sentences(x)

In [11]:
from gensim.models import word2vec

#配置词向量的维度
num_features=300
#保证被考虑的词汇的频度
min_word_count=20
#设定并行化训练使用CPU计算核心的数量，多核可用
num_workers=2
#定义训练词向量的上下文窗口大小
context=5
downsampling=1e-3#1e-3第一个是数字1，不是字母l

from gensim.models import word2vec

#训练词向量模型
model=word2vec.Word2Vec(sentences,workers=num_workers,\
                       size=num_features,min_count=min_word_count,\
                       window=context,sample=downsampling)

#这个设定代表当前训练好的词向量为最终版，也可以加快模型的训练速度
model.init_sims(replace=True)

In [12]:
#利用训练好的模型，寻找训练文本中与morning最相关的10个词汇
model.most_similar('morning')

d:\ai\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('afternoon', 0.8176407814025879),
 ('weekend', 0.7953598499298096),
 ('evening', 0.7908556461334229),
 ('saturday', 0.7033148407936096),
 ('night', 0.702003002166748),
 ('friday', 0.6876775026321411),
 ('sunday', 0.685408353805542),
 ('newspaper', 0.6757447123527527),
 ('week', 0.6473132371902466),
 ('summer', 0.6368684768676758)]

In [13]:
model.most_similar('email')

d:\ai\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('mail', 0.7279140949249268),
 ('contact', 0.675162672996521),
 ('request', 0.6692183017730713),
 ('address', 0.6512033939361572),
 ('snail', 0.64162278175354),
 ('send', 0.6322551965713501),
 ('subscription', 0.6288837194442749),
 ('listserv', 0.6252148151397705),
 ('sas', 0.6230258941650391),
 ('replies', 0.6117781400680542)]

In [ ]:
# 在不用语言学词典的情况下，词向量技术可以借助上下文来找到词汇之间的相似性
